In [ ]:
!export CUDA_VISIBLE_DEVICES=0

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as weight_init
from torch.autograd import Variable
import torch.optim as optim
import torch.backends.cudnn as cudnn

import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
class netG(nn.Module):
    def __init__(self, nz, ngf, nc):
        super(netG, self).__init__()
               
        self.convT_1 = nn.ConvTranspose2d(nz, ngf*8, 4, 1, 0, bias=False)
        self.bn1 = nn.BatchNorm2d(ngf*8)
        
        self.convT_2 = nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(ngf*4)
        
        self.convT_3 = nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(ngf*2)
        
        self.convT_4 = nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False)
        self.bn4 = nn.BatchNorm2d(ngf)
        
        self.convT_5 = nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False)
        
        for m in self.modules():
            if isinstance(m, nn.ConvTranspose2d):
                weight_init.kaiming_normal(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.normal_(1.0, 0.01)
                m.bias.data.fill_(0)

    def forward(self, input):
        out = F.relu(self.bn1(self.convT_1(input)))
        out = F.relu(self.bn2(self.convT_2(out)))
        out = F.relu(self.bn3(self.convT_3(out)))
        out = F.relu(self.bn4(self.convT_4(out)))
        out = F.tanh(self.convT_5(out))
        return out

In [ ]:
class netD(nn.Module):
    def __init__(self, nc, ndf, nz):
        super(netD, self).__init__()
        
        self.conv1 = nn.Conv2d(nc, ndf, 4, 2, 1, bias=False)
        
        self.conv2 = nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(ndf*2)
        
        self.conv3 = nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(ndf*4)
        
        self.conv4 = nn.Conv2d(ndf*4, ndf*8, 4, 2, 1, bias=False)
        self.bn4 = nn.BatchNorm2d(ndf*8)
        
        self.conv5 = nn.Conv2d(ndf*8, 1, 4, 1, 0, bias=False)
        
        for m in self.modules():
            if isinstance(m, nn.ConvTranspose2d):
                weight_init.kaiming_normal(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.normal_(1.0, 0.01)
                m.bias.data.fill_(0)
        
    def forward(self, input):
        out = F.leaky_relu(self.conv1(input), 0.2, inplace=True)
        
        out = self.bn2(self.conv2(out))
        out = F.leaky_relu(out, 0.2, inplace=True)
        
        out = self.bn3(self.conv3(out))
        out = F.leaky_relu(out, 0.2, inplace=True)

        out = self.bn4(self.conv4(out))
        out = F.leaky_relu(out, 0.2, inplace=True)
        
        out = self.conv5(out)
        
        out = F.sigmoid(out)
        out =  out.view(-1, 1).squeeze(1)
        return out

In [ ]:
#opts
LR = 0.0002
beta1 = 0.5

NITER = 100
BATCH_SIZE = 64

nz = int(100)
ngf = int(64)
ndf = int(64)
nc = 3

outf = './'
DATA_ROOT = './data'
try:
    os.makedirs(outf)
except: pass

cudnn.benchmark = True
torch.manual_seed(0)
torch.cuda.manual_seed(0)

In [ ]:
dataset = dset.MNIST(root=DATA_ROOT, download=True,
                           transform=transforms.Compose([
                               transforms.Scale(64),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]))

# dataset = dset.LSUN(db_path=DATA_ROOT, classes=['bedroom_train'],
#                         transform=transforms.Compose([
#                             transforms.Scale(64),
#                             transforms.CenterCrop(64),
#                             transforms.ToTensor(),
#                             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
# ]))

In [ ]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE,
                                         shuffle=True, num_workers=4)

In [ ]:
G = netG(nz, ngf, nc).cuda()
D = netD(nc, ndf, nz).cuda()
criterion = nn.BCELoss().cuda()

noise = torch.FloatTensor(BATCH_SIZE, nz, 1, 1).cuda()
fixed_noise = Variable(torch.FloatTensor(BATCH_SIZE, nz, 1, 1).normal_(0, 1)).cuda()

real_label = 1
fake_label = 0

In [ ]:
optimizer_G = optim.Adam(G.parameters(), lr=LR, betas=(beta1 ,0.999))
optimizer_D = optim.Adam(D.parameters(), lr=LR, betas=(beta1 ,0.999))

In [ ]:
for epoch in range(NITER):
    for i, data in enumerate(dataloader):
        # Discriminator
        D.zero_grad()
        
        # Discriminator Feed real data
        real, _ = data
        batch_size = real.size(0)
        
        real = Variable(real).cuda()
        label = Variable(torch.Tensor(batch_size).fill_(real_label)).cuda()
        
        output = D(real)
        errD_real = criterion(output, label)
        errD_real.backward()
        
        D_x = output.data.mean()
        
        # Feed fake data
        noise.resize_(batch_size, nz, 1, 1).normal_(0,1)
        noisev = Variable(noise)
        label = Variable(torch.Tensor(batch_size).fill_(fake_label)).cuda()
        
        fake = G(noisev)
        output = D(fake.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        
        D_G_z1 = output.data.mean()
        
        errD = errD_real + errD_fake
        optimizer_D.step()
        
        # Generator
        G.zero_grad()
        label = Variable(torch.Tensor(batch_size).fill_(real_label)).cuda()
        output = D(fake)
        errG = criterion(output, label)
        errG.backward()
        optimizer_G.step()
        D_G_z2 = output.data.mean()
        
# if i%100 == 0:
    print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
          % (epoch, NITER, i, len(dataloader),
             errD.data[0], errG.data[0], D_x, D_G_z1, D_G_z2))

    vutils.save_image(real.data.cpu(), '%s/real_samples.png' % outf,
                                    normalize=True)
    fake = G(fixed_noise)
    vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % (outf, i),
                                    normalize=True)

In [ ]:
dummy = Variable(torch.Tensor(1,nz,1,1).normal_(0,1)).cuda()
out = G(dummy) 
# out = D(out)
# print out
out = out.squeeze()
out = out.data.cpu().numpy()

plt.imshow(out, cmap='Greys')
# plt.imshow(np.transpose(out, [1,2,0]))